In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline
import matplotlib.pyplot as plt

# always import gbm_algos first !
# import xgboost, lightgbm, catboost
from gplearn.genetic import SymbolicRegressor

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from preprocess import *

# Let's do facy optimizers
from optimizers import Lookahead, AdamGC, SGDGC
from madgrad import MADGRAD
from lbfgsnew import LBFGSNew
# Modify at /usr/local/lib/python3.9/site-packages/torch_lr_finder/lr_finder.py
from torch_lr_finder import LRFinder

# Distance loss
from geomloss import SamplesLoss

from tqdm import trange

[pyKeOps]: Warning, no cuda detected. Switching to cpu only.


In [2]:
include_N_res = False

DATA_PATH = '../PDE_FIND_experimental_datasets/kuramoto_sivishinky.mat'
X, T, Exact = space_time_grid(data_path=DATA_PATH, real_solution=True)
X_star, u_star = get_trainable_data(X, T, Exact)

# Domain bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

N = 10000
print(f"Training with {N} samples")

# idx = np.random.choice(X_star.shape[0], N, replace=False)
idx = np.arange(N) # arange for faster training due to the easy data

X_u_train = X_star[idx, :]
u_train = u_star[idx,:]

# Unsup data
if include_N_res:
    N_res = N//2
    idx_res = np.array(range(X_star.shape[0]-1))[~idx]
    idx_res = np.random.choice(idx_res.shape[0], N_res, replace=True)
    X_res = X_star[idx_res, :]
    print(f"Training with {N_res} unsup samples")
    X_u_train = np.vstack([X_u_train, X_res])
    u_train = np.vstack([u_train, torch.rand(X_res.shape[0], 1) - 1000])
    # del X_res
else: print("Not including N_res")
    
# Convert to torch.tensor
X_u_train = to_tensor(X_u_train, True)
u_train = to_tensor(u_train, False)
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)

# lb and ub are used in adversarial training
scaling_factor = 1.0
lb = scaling_factor*to_tensor(lb, False)
ub = scaling_factor*to_tensor(ub, False)

# Feature names
feature_names=('uf', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx')

Training with 10000 samples
Not including N_res


In [3]:
fdc = FinDiffCalculator(X, T, Exact, acc_order=10)
fd_u_t = to_tensor(fdc.finite_diff(1, diff_order=1), False)
fd_derivatives = fdc.finite_diff_from_feature_names(feature_names)
for d in fd_derivatives: fd_derivatives[d] = to_tensor(fd_derivatives[d], False)

dx = 0.09817477042468103
dt = 0.4


In [4]:
tmp = None
for f in feature_names:
    print('Concatenating', f)
    if tmp == None: tmp = fd_derivatives[f]
    else: tmp = torch.cat([tmp, fd_derivatives[f]], dim=-1)
fd_derivatives = tmp[:, :]
(((fd_u_t+fd_derivatives[:, 4:5]+(fd_derivatives[:, 0:1]*fd_derivatives[:, 1:2])+fd_derivatives[:, 2:3]))**2).mean().item()

Concatenating uf
Concatenating u_x
Concatenating u_xx
Concatenating u_xxx
Concatenating u_xxxx


1.4235734852263704e-05

In [5]:
class Network(nn.Module):
    def __init__(self, model, index2features=None, scale=False, lb=None, ub=None):
        super(Network, self).__init__()
        # pls init the self.model before
        self.model = model
        # For tracking, the default tup is for the burgers' equation.
        self.index2features = index2features
        print("Considering", self.index2features)
        self.diff_flag = diff_flag(self.index2features)
        self.uf = None
        self.scale = scale
        self.lb, self.ub = lb, ub
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def forward(self, x, t):
        if not self.scale: self.uf = self.model(torch.cat([x, t], dim=1))
        else: self.uf = self.model(self.neural_net_scale(torch.cat([x, t], dim=1)))
        return self.uf
    
    def get_selector_data(self, x, t):
        uf = self.forward(x, t)
        u_t = self.gradients(uf, t)[0]
        
        ### PDE Loss calculation ###
        # Without calling grad
        derivatives = []
        for t in self.diff_flag[0]:
            if t=='uf': derivatives.append(uf)
            elif t=='x': derivatives.append(x)
        # With calling grad
        for t in self.diff_flag[1]:
            out = uf
            for c in t:
                if c=='x': out = self.gradients(out, x)[0]
                elif c=='t': out = self.gradients(out, t)[0]
            derivatives.append(out)
        
        return torch.cat(derivatives, dim=1), u_t
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))
    
    def neural_net_scale(self, inp):
        return 2*(inp-self.lb)/(self.ub-self.lb)-1

In [6]:
class SeclectorNetwork(nn.Module):
    def __init__(self, X_train_dim, bn=None):
        super().__init__()
        # Nonlinear model, Training with PDE reg.
        self.nonlinear_model = TorchMLP(dimensions=[X_train_dim, 50, 50, 1], activation_function=nn.Tanh, bn=bn, dropout=nn.Dropout(p=0.1), inp_drop=False)
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def forward(self, inn):
        ut_approx = self.nonlinear_model(inn)
        return ut_approx
    
    def loss(self, X_input, y_input):
        ut_approx = self.forward(X_input)
        mse_loss = F.mse_loss(ut_approx, y_input, reduction='mean')
        return mse_loss


class AttentionSelectorNetwork(nn.Module):
    def __init__(self, layers, prob_activation=torch.sigmoid, bn=None, reg_intensity=0.25):
        super(AttentionSelectorNetwork, self).__init__()
        # Nonlinear model, Training with PDE reg.
        assert len(layers) > 1
        self.linear1 = nn.Linear(layers[0], layers[0])
        self.prob_activation = prob_activation
        self.nonlinear_model = TorchMLP(dimensions=layers, activation_function=nn.Tanh, bn=bn, dropout=nn.Dropout(p=0.1))
        self.latest_weighted_features = None
        self.th = 1/layers[0]
        self.reg_intensity = reg_intensity
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        
    def forward(self, inn):
        return self.nonlinear_model(inn*self.weighted_features(inn))
    
    def weighted_features(self, inn):
        self.latest_weighted_features = self.prob_activation(self.linear1(inn)).mean(axis=0)
        return self.latest_weighted_features
    
    def loss(self, X_input, y_input):
        ut_approx = self.forward(X_input)
        mse_loss = F.mse_loss(ut_approx, y_input, reduction='mean')
        return mse_loss+self.reg_intensity*torch.norm(F.relu(self.latest_weighted_features-self.th), p=0)

# Only the SemiSupModel has changed to work with the finite difference guidance
class SemiSupModel(nn.Module):
    def __init__(self, network, selector, normalize_derivative_features=False, mini=None, maxi=None):
        super(SemiSupModel, self).__init__()
        self.network = network
        self.selector = selector
        self.normalize_derivative_features = normalize_derivative_features
        self.mini = mini
        self.maxi = maxi
        
    def forward(self, X_u_train, fd_derivatives=None, fd_u_t=None, fd_weight=0.0, include_unsup=True):
        X_selector, y_selector = self.network.get_selector_data(*dimension_slicing(X_u_train))
        
        fd_guidance = 0.0
        if fd_weight>0.0 and fd_derivatives is not None and fd_u_t is not None:
            # Traditional MSE Loss btw uf and u_train + the fd_guidance loss
            row, col = fd_derivatives.shape
            fd_guidance += F.mse_loss(X_selector[:row, 0:1], fd_derivatives[:, 0:1])
            fd_guidance += fd_weight*(col-1)*F.mse_loss(X_selector[:row, 1:], fd_derivatives[:, 1:])
            fd_guidance += fd_weight*F.mse_loss(y_selector[:row, :], fd_u_t)
            
        else: fd_guidance = self.network.uf
            
        if self.normalize_derivative_features:
            X_selector = (X_selector-self.mini)/(self.maxi-self.mini)
        
        if include_unsup: unsup_loss = self.selector.loss(X_selector, y_selector)
        else: unsup_loss = None
        
        return fd_guidance, unsup_loss

In [7]:
### Version with normalized derivatives ###
# pretrained_state_dict = torch.load("./saved_path_inverse_ks/pretrained5000samples_semisup_model_with_LayerNormDropout_without_physical_reg.pth")
pretrained_state_dict = cpu_load("./saved_path_inverse_ks/semisup_model_with_LayerNormDropout_without_physical_reg_trained10000labeledsamples_trained0unlabeledsamples.pth")
# pretrained_state_dict = None
use_pretrained_weights = True
lets_pretrain = not use_pretrained_weights

semisup_model = SemiSupModel(network=Network(
                                    model=TorchMLP(dimensions=[2, 50, 50, 50 ,50, 50, 1],
                                                   activation_function=nn.Tanh,
                                                   bn=nn.LayerNorm, dropout=None),
                                    index2features=feature_names, scale=True, lb=lb, ub=ub),
                            selector=AttentionSelectorNetwork([len(feature_names), 50, 50, 1], prob_activation=nn.Softmax(dim=1), bn=nn.LayerNorm),
                            normalize_derivative_features=True,
                            mini=None,
                            maxi=None)

if use_pretrained_weights:
    print("Use pretrained weights")
    semisup_model.load_state_dict(pretrained_state_dict)
#     referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_star))
#     semisup_model.mini = torch.min(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)
#     semisup_model.maxi = torch.max(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)
    semisup_model.mini = tmp.min(axis=0)[0].requires_grad_(False)
    semisup_model.maxi = tmp.max(axis=0)[0].requires_grad_(False)

# Highly inefficient code here
# Why I cannnot load the network state_dict inside the if statement???
tmp = SemiSupModel(network=Network(
                                    model=TorchMLP(dimensions=[2, 50, 50, 50 ,50, 50, 1],
                                                   activation_function=nn.Tanh,
                                                   bn=nn.LayerNorm, dropout=None),
                                    index2features=feature_names, scale=True, lb=lb, ub=ub),
                            selector=AttentionSelectorNetwork([len(feature_names), 50, 50, 1], prob_activation=nn.Softmax(dim=1), bn=nn.LayerNorm),
                            normalize_derivative_features=True,
                            mini=None,
                            maxi=None)
tmp.load_state_dict(pretrained_state_dict); network_state_dict = tmp.network.state_dict()
del tmp, pretrained_state_dict

Using old implementation of TorchMLP. See models.py for more new model-related source code.
Considering ('uf', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx')
Using old implementation of TorchMLP. See models.py for more new model-related source code.
Use pretrained weights
Using old implementation of TorchMLP. See models.py for more new model-related source code.
Considering ('uf', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx')
Using old implementation of TorchMLP. See models.py for more new model-related source code.


In [8]:
# Chk the performance both MSE and PDE relation loss
semisup_model.eval()
print(F.mse_loss(semisup_model.network(*dimension_slicing(X_star)).detach(), u_star).item())
referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_u_train))
(((u_t+referenced_derivatives[:, 4:5]+(referenced_derivatives[:, 0:1]*referenced_derivatives[:, 1:2])+referenced_derivatives[:, 2:3]))**2).mean().item()

10.407444953918457


1.013510518532712e-05

In [9]:
if lets_pretrain:
    print("Pretraining")
    pretraining_optimizer = LBFGSNew(semisup_model.network.parameters(), 
                                     lr=1e-1, max_iter=300, 
                                     max_eval=int(300*1.25), history_size=150, 
                                     line_search_fn=True, batch_mode=False)
#     pretraining_optimizer =  MADGRAD(semisup_model.network.parameters(), lr=1e-3, momentum=0.9)
    
    best_state_dict = None; curr_loss = 1000
    semisup_model.network.train()
    for i in range(300):
        def pretraining_closure():
            global N, X_u_train, u_train
            if torch.is_grad_enabled():
                pretraining_optimizer.zero_grad()
            # Only focusing on first [:N, :] elements
            mse_loss = F.mse_loss(semisup_model.network(*dimension_slicing(X_u_train[:N, :])), u_train[:N, :])
            if mse_loss.requires_grad:
                mse_loss.backward(retain_graph=False)
            return mse_loss

        pretraining_optimizer.step(pretraining_closure)

        l = pretraining_closure()
        
        if l.item() < curr_loss:
            curr_loss = l.item()
            best_state_dict = semisup_model.state_dict()
            
        if (i%10)==0:
            print("Epoch {}: ".format(i), curr_loss)

            # Sneak on the test performance...
            semisup_model.network.eval()
            test_performance = F.mse_loss(semisup_model.network(*dimension_slicing(X_star)).detach(), u_star).item()
            string_test_performance = scientific2string(test_performance)
            print('Test MSE:', string_test_performance)
    
    print("Computing derivatives features")
    semisup_model.eval()
    referenced_derivatives, _ = semisup_model.network.get_selector_data(*dimension_slicing(X_star))
    
#     semisup_model.mini = torch.min(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)
#     semisup_model.maxi = torch.max(referenced_derivatives, axis=0)[0].detach().requires_grad_(False)

    semisup_model.mini = tmp.min(axis=0)[0].requires_grad_(False)
    semisup_model.maxi = tmp.max(axis=0)[0].requires_grad_(False)

In [10]:
def pcgrad_closure():
    global N, X_u_train, u_train, fd_derivatives, fd_u_t
    fd_guidance, unsup_loss = semisup_model(X_u_train, fd_derivatives=fd_derivatives, 
                                            fd_u_t=fd_u_t, fd_weight=1.0, include_unsup=True)
    losses = [fd_guidance, unsup_loss]
    updated_grads = []
    
    for i in range(2):
        optimizer1.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in semisup_model.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(semisup_model.parameters()):
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    return sum(losses)

def selector_closure():
    global ders, dys
    if torch.is_grad_enabled():
        optimizer3.zero_grad()
    unsup_loss = semisup_model.selector.loss(ders, dys)
    if unsup_loss.requires_grad:
        unsup_loss.backward(retain_graph=True)
    return unsup_loss

In [11]:
params = semisup_model.parameters()

### For SGD and Adam ###
learning_rate1, learning_rate2 = 1e-7, 1e-1

### For LBFGS (a good choice already!!!) ###
# print("Using LBFGS's learning rate set")
# learning_rate1, learning_rate2 = 8e-2, 5e-2 # (1e-1, 5e-2) is also OK!

choice = 'MAD'; auto_lr = False
if choice == 'LBFGS':
    optimizer1 = torch.optim.LBFGS(params, lr=learning_rate1, 
                                   max_iter=100, max_eval=125, 
                                  history_size=120, line_search_fn='strong_wolfe')
if choice == 'Adam':
    optimizer1 = AdamGC(params, lr=learning_rate1, use_gc=True, gc_conv_only=False, gc_loc=False)
if choice == 'SGD':
    optimizer1 = SGDGC(params, lr=learning_rate1, use_gc=True, nesterov=True, momentum=0.95)
if choice == 'MAD':
    optimizer1 = MADGRAD(params, lr=learning_rate1, momentum=0.9)

if choice != 'LBFGS' and auto_lr:
    print('Learning rate finding')
    bs = 4000; bs = X_u_train.shape[0] if bs>X_u_train.shape[0] else bs
    criterion = LadderLoss(return_list=True)
    trainloader = get_dataloader(X_u_train, u_train, bs=bs)
    
    lr_finder = LRFinder(semisup_model, optimizer=optimizer1, 
                         closure=pcgrad_update, criterion=criterion, device="cpu")
    lr_finder.range_test(trainloader, val_loader=None, end_lr=100, num_iter=300)
    
    # to inspect the loss-learning rate graph
    suggested_lr, _ = lr_finder.plot()
    # To prevent divergence during the second stage training.
    # suggested_lr = min(suggested_lr, 5e-3)
    lr_finder.reset(); plt.show()

else:
    lr_finder = None
    suggested_lr = None

In [12]:
print("Deleted the fake labels used in Learning rate finder")
u_train = u_train[:N, :]

suggested_lr = 1e-6

if lr_finder and suggested_lr:
    optimizer1 = lr_finder.optimizer

print("Assigning the suggested_lr to optimizer1")
for g in optimizer1.param_groups:
    g['lr'] = suggested_lr

epochs1 = 300; epochs2 = 100;

Deleted the fake labels used in Learning rate finder
Assigning the suggested_lr to optimizer1


In [13]:
# # Setting up the generator
# generator = TorchMLP([2, 50, 50, 2])
# # generator_training_epochs indicates how string the generator is
# adv_f = 100; generator_training_epochs = 300; generator_training_limit = epochs1-100
# # I can use the Learning rate finder to find a good lr for the generator optim  as well
# generator_optimizer = torch.optim.SGD(generator.parameters(), lr=3e-4, momentum=0.95)
# # sinkhorn distance loss
# sinkhorn_loss = SamplesLoss("sinkhorn", p=1, blur=10)

In [14]:
# curr_loss = 500; F_print = 10 if choice == 'LBFGS' else 100

# # Stage I
# # How long should I pretrain selector part of the model?
# for i in range(epochs1):
#     if i%adv_f==0 and i<=generator_training_limit:
#         best_generator_loss = 1e6; best_generator_state_dict = None
#         o_tensor = X_u_train[:N, :] # or X_u_train ?
#         print("Training the generator for ")
#         for _ in trange(generator_training_epochs):
#             semisup_model.eval()
#             generator.train()
#             generator_optimizer.zero_grad()
            
#             # Do I need to scale o_tensor before feeding into the generator?
#             X_gen = scale_to_range(generator(o_tensor), lb, ub)
#             unsup_loss = semisup_model(X_gen)[1]
            
#             # Choose the distance function that works well with the X_u_train structure
#             d_loss = sinkhorn_loss(X_gen, o_tensor)
# #             d_loss = F.mse_loss(X_gen, o_tensor)

#             generator_loss = 0.05*d_loss-unsup_loss
#             generator_loss.backward(retain_graph=True)
#             generator_optimizer.step()
            
#             # Saving the best_generator_state_dict
#             if generator_loss.item() < best_generator_loss:
#                 best_generator_loss = generator_loss.item()
#                 best_generator_state_dict = generator.state_dict()

#         print("The best generator loss:", best_generator_loss)
#         if best_generator_state_dict is not None: 
#             generator.load_state_dict(best_generator_state_dict)
            
#         generator.eval()
#         X_gen = scale_to_range(generator(o_tensor), lb, ub)
#         if N_res<X_gen.shape[0]: X_gen = sampling_from_rows(X_gen, N_res)
#         X_u_train = torch.cat([X_u_train[:N, :], X_gen], dim=0).detach().requires_grad_(True)
        
#         # Re init
#         with torch.no_grad(): semisup_model.network.load_state_dict(network_state_dict)

#     semisup_model.train()
#     generator_optimizer.zero_grad()
#     optimizer1.step(pcgrad_closure)
#     l = pcgrad_closure()
#     if (i % F_print) == 0:
#         if l.item() != curr_loss:
#             curr_loss = l.item()
#         else:
#             print("Semi-supervised solver loss @Epoch {}: ".format(i), curr_loss)
#             print("Finishing the first stage")
#             break
#         print("Semi-supervised solver loss @Epoch {}: ".format(i), curr_loss)

In [15]:
# # Stage II: Train semisup_model.network
# def closure():
#     global N, X_u_train, u_train, fd_derivatives, fd_u_t
#     if torch.is_grad_enabled():
#         optimizer2.zero_grad()
#     # With fd guidance
#     mse_loss = semisup_model(X_u_train, fd_derivatives, fd_u_t, 2.0, False)[0]
#     if mse_loss.requires_grad:
#         mse_loss.backward(retain_graph=True)
#     return mse_loss

# # optimizer2 = torch.optim.LBFGS(semisup_model.network.parameters(), 
# #                                      lr=1e-1, max_iter=100, 
# #                                      max_eval=int(100*1.25), history_size=150, 
# #                                      line_search_fn='strong_wolfe')

# optimizer2 = MADGRAD(semisup_model.network.parameters(), lr=1e-6, momentum=0.95)

# curr_loss = 10000
# semisup_model.network.train()
# semisup_model.selector.eval()
# for i in range(epochs2):
#     optimizer2.step(closure)
#     if (i % 2) == 0:
#         l = closure()
#         if l.item() != curr_loss:
#             curr_loss = l.item()
#         print("Epoch {}: ".format(i), curr_loss)

In [16]:
# torch.save(semisup_model.state_dict(), "../inverse_KS/saved_path_inverse_ks/semisup_model_with_LayerNormDropout_without_physical_reg_trained7500labeledsamples_trained0unlabeledsamples.pth")

In [17]:
fd_derivatives = to_numpy(fd_derivatives)
fd_u_t = to_numpy(fd_u_t)

In [18]:
n_test = 12000
referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_star[:n_test, :]))
referenced_derivatives = to_numpy(referenced_derivatives); u_t = to_numpy(u_t)
(((u_t+referenced_derivatives[:, 4:5]+(referenced_derivatives[:, 0:1]*referenced_derivatives[:, 1:2])+referenced_derivatives[:, 2:3]))**2).mean().item()

2.3217648049467243e-05

In [19]:
from sklearn.preprocessing import PolynomialFeatures
alpha = 1
const_range = (-1.5, 1.5)

X_input = alpha*referenced_derivatives+(1-alpha)*fd_derivatives[:n_test, :]
y_input = alpha*u_t+(1-alpha)*fd_u_t[:n_test, :]

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_input = poly.fit_transform(X_input)
poly_feature_names = poly.get_feature_names(feature_names)
for i, f in enumerate(poly_feature_names):
    poly_feature_names[i] = f.replace(" ", "*")

from sparsereg.model import STRidge
# print(STRidge(unbias=False).fit(fd_derivatives, fd_u_t.ravel()).coef_)
coef = STRidge(threshold=0.01, alpha=100, max_iter=1000, normalize=True).fit(X_input, y_input.ravel()).coef_
idxs = np.argsort(np.abs(coef))[::-1]

# print(coef)
# idxs = np.nonzero(coef)[0]

for idx in idxs[:]:
    if not np.isclose(coef[idx], 0.0) and coef[idx]<const_range[1] and coef[idx]>const_range[0]:
        print(str(coef[idx])+poly_feature_names[idx], end="")

-1.1236142274555205u_xx-1.0062766635476028uf*u_x-0.31973993926315875u_xxxx

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# Test on 2048 samples => 2.0e-6
n_test_samples = 2048

print("Testing")
semisup_model.network.eval()
# Compare btw the two semi-supervise learning?
test_performance = F.mse_loss(semisup_model.network(*dimension_slicing(X_star[:n_test_samples, :])).detach(), u_star[:n_test_samples, :]).item()
string_test_performance = scientific2string(test_performance)
print('Test MSE:', string_test_performance)

In [ ]:
ders, dys = semisup_model.network.get_selector_data(*dimension_slicing(X_u_train))
ders = (ders-semisup_model.mini)/(semisup_model.maxi-semisup_model.mini)

optimizer3 = LBFGSNew(semisup_model.selector.parameters(),
                      lr=learning_rate2, max_iter=100, max_eval=int(100*1.25),
                      history_size=150, line_search_fn=True, batch_mode=False)

# Stage II: Train semisup_model.selector
semisup_model.network.eval()
semisup_model.selector.train()
for i in range(20):
    optimizer3.step(selector_closure)
    if (i % 5) == 0:
        l = selector_closure()
        print("Epoch {}: ".format(i), l.item())
        print(np.argsort((semisup_model.selector.latest_weighted_features).detach().numpy()))

In [ ]:
feature_importance = (semisup_model.selector.latest_weighted_features).detach().numpy()
print("--- Feature importance ranking ---")
for idx in np.argsort(feature_importance)[::-1]:
    print(feature_names[idx], feature_importance[idx])

In [ ]:
light = lightgbm.LGBMRegressor(n_estimators=300, learning_rate=0.1, reg_lambda=1)
light = SklearnModel(model=light, X_train=to_numpy(ders), y_train=to_numpy(dys).ravel(), feature_names=feature_names)
light_feature_importance = light.feature_importance()

In [ ]:
idx = np.arange(20000)
referenced_derivatives, u_t = semisup_model.network.get_selector_data(*dimension_slicing(X_star[idx, :]))
(((u_t+referenced_derivatives[:, 4:5]+(referenced_derivatives[:, 0:1]*referenced_derivatives[:, 1:2])+referenced_derivatives[:, 2:3]))**2).mean().item()

In [ ]:
est_gp = SymbolicRegressor(population_size=60000, generations=3, tournament_size=50,
                           function_set=('add', 'sub', 'mul'), const_range=(-1, 1),
                           p_crossover=0.7, p_subtree_mutation=0.1, p_hoist_mutation=0.05,
                           p_point_mutation=0.1, parsimony_coefficient=0.001, max_samples=0.9,
                           verbose=1, low_memory=False, n_jobs=3)
est_gp.fit(referenced_derivatives, u_t.ravel())

In [ ]:
build_exp(est_gp._program)

In [ ]:
from pysr import pysr, best, best_callable

In [ ]:
equations = pysr(to_numpy(referenced_derivatives), to_numpy(u_t).ravel(), niterations=5, 
                 binary_operators=["-", "*"], unary_operators=[], procs=3, 
                 populations=20, npop=4000)